In [31]:
import sys
sys.path.insert(0, '../QuantumLattice')
from QuantumLattice import QuantumLattice

#Import QuantumLattice and test.
u = [1,2,3]
N = 11
t = [[10,4,1],[4,10,5],[1,5,10]]
M = 3

entity1 = QuantumLattice(latticeLength = N,numParticleType = M)
entity2 = QuantumLattice(latticeLength = N,numParticleType = M)

child = entity1.crossover(entity2)

child.mutate()

child.evaluateFitness(selfEnergyVector = u,interactionEnergyMatrix= t)
print(child)


---------------------------------
Child's INFO:
Lattice is : [2, 0, 1, 0, 2, 1, 1, 2, 2, 0, 0]
Total energy:680
LatticeLength:11
NumOfParticleTypes: 3
Sigma = 8.39914075e-01


In [32]:
#
# Population.py
#
#

import copy
import math
from operator import attrgetter
import random

from numpy import Inf, Infinity
sys.path.insert(0, '../')

class Population:
    """
    Population
    """
    # uniprng=None
    crossoverFraction=None
    minEnergy = 0

    def __init__(self, 
                 populationSize,
                 latticeLength,
                 numParticleType,
                 selfEnergyVector,
                 interactionEnergyMatrix,
                 crossoverFraction):
        """
        Population constructor
        """
        self.population=[]
        self.selfEnergyVector = selfEnergyVector
        self.interactionEnergyMatrix = interactionEnergyMatrix
        self.crossoverFraction = crossoverFraction
        self.minEnergyLattice = []
        
        for i in range(populationSize):
            self.population.append(QuantumLattice(latticeLength = latticeLength,numParticleType = numParticleType))
            

    def __len__(self):
        return len(self.population)

    def __getitem__(self,key):
        return self.population[key]

    def __setitem__(self,key,newValue):
        self.population[key]=newValue

    def copy(self):
        return copy.deepcopy(self)

    def evaluateFitness(self):
        #Finding the minimum Energy of Pop and Calculate the Energy for every individuals
        energy_distribution = []
        
        for individual in self.population: 
            individual.evaluateFitness(self.selfEnergyVector,
                                       self.interactionEnergyMatrix)
            energy_distribution.append(individual.energy)
            
        self.minEnergy = min(energy_distribution)
        minEnergy_idx = energy_distribution.index(self.minEnergy)
        
        self.minEnergyLattice = self.population[minEnergy_idx].lattice
        
    # def mutate(self):
        # for individual in self.population:
            # individual.mutate()

    def crossover(self):
        children = []
        indexList1=list(range(len(self)))
        indexList2=list(range(len(self)))
        random.shuffle(indexList1)
        random.shuffle(indexList2)

        #Generate children
        if self.crossoverFraction == 1.0:
            for index1,index2 in zip(indexList1,indexList2):
                childx = self[index1].crossover(self[index2])
                childx.mutate()
                children.append(childx)
        else:
            for index1,index2 in zip(indexList1,indexList2):
                rn=random.random()
                if rn < self.crossoverFraction:
                    childx = self[index1].crossover(self[index2])
                    childx.mutate()
                    children.append(childx)
        
        #Children has to be evaluated
        for child in children:
            child.evaluateFitness(selfEnergyVector=self.selfEnergyVector,
                                  interactionEnergyMatrix=self.interactionEnergyMatrix)
        
        #Replace some parents.
        for childx in children:
            for idx,parent in enumerate(self.population):
                if childx.energy < parent.energy:
                    self.population[idx] = childx
                    break

    def conductTournament(self):
        # binary tournament, by finding the minEnergy.
        indexList1=list(range(len(self.population)))
        indexList2=list(range(len(self.population)))

        random.shuffle(indexList1)
        random.shuffle(indexList2)

        # do not allow self competition
        for i in range(len(self.population)):
            if indexList1[i] == indexList2[i]:
                temp=indexList2[i]
                if i == 0:
                    indexList2[i]=indexList2[-1]
                    indexList2[-1]=temp
                else:
                    indexList2[i]=indexList2[i-1]
                    indexList2[i-1]=temp

        #compete
        newPop=[]
        for index1,index2 in zip(indexList1,indexList2):
            if self.population[index1].energy < self.population[index2].energy:
                newPop.append(copy.deepcopy(self.population[index1]))
            elif self.population[index1].energy > self.population[index2].energy:
                newPop.append(copy.deepcopy(self.population[index2]))
            else:
                rn=random.random()
                if rn > 0.5:
                    newPop.append(copy.deepcopy(self.population[index1]))
                else:
                    newPop.append(copy.deepcopy(self.population[index2]))

        # overwrite old pop with newPop
        self.population=newPop


    # def combinePops(self,otherPop):
        # self.population.extend(otherPop.population)

    def truncateSelect(self,newPopSize):
        #sort by fitness
        self.population.sort(key=attrgetter('energy'),reverse=True)

        #then truncate the bottom
        self.population=self.population[:newPopSize]

    def __str__(self):
        print("------------------------------\nPopulation [INFO]")
        print(f"\nInteractionMatrix : \n {self.interactionEnergyMatrix}")
        print(f"\nEnergyVector : \n {self.selfEnergyVector}")
        print(f"\nPopulation:\n")
        for individual in self.population:
            print(individual.lattice)
        print(f"\nMinEnergy : \n {self.minEnergy} with lattice {self.minEnergyLattice}")
    
        return "\n" 
        

In [33]:
#Test Pop
u = [1,2] #SelfEnergyVector
N = 5 #Lattice length
P = 10 #PopSize
t = [[3,5],[5,3]] #InteractionMatrix
M = 2 #NumOfTypesOfParticles
crossoverfraction = 0.2

Pop = Population(populationSize = P,
                 latticeLength = N,
                 numParticleType = M,
                 selfEnergyVector = u,
                 interactionEnergyMatrix = t,
                 crossoverFraction = crossoverfraction)
Pop.evaluateFitness()

print(Pop)
Pop.crossover()
Pop.evaluateFitness()
print(Pop)
#Test CrossOver
Pop.crossover()
Pop.evaluateFitness()
print(Pop)


------------------------------
Population [INFO]

InteractionMatrix : 
 [[3, 5], [5, 3]]

EnergyVector : 
 [1, 2]

Population:

[1, 0, 1, 1, 0]
[0, 1, 1, 0, 0]
[1, 1, 0, 0, 1]
[1, 0, 1, 1, 0]
[1, 1, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 1, 1, 0, 1]
[0, 1, 0, 1, 0]
[1, 1, 1, 1, 0]
[1, 1, 1, 1, 0]

MinEnergy : 
 97 with lattice [1, 0, 0, 0, 0]


------------------------------
Population [INFO]

InteractionMatrix : 
 [[3, 5], [5, 3]]

EnergyVector : 
 [1, 2]

Population:

[0, 1, 1, 1, 1]
[0, 1, 1, 0, 0]
[1, 1, 0, 0, 1]
[1, 0, 1, 1, 0]
[1, 1, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 1, 1, 0, 1]
[0, 1, 0, 1, 0]
[1, 1, 1, 1, 0]
[1, 1, 1, 1, 0]

MinEnergy : 
 97 with lattice [1, 0, 0, 0, 0]


------------------------------
Population [INFO]

InteractionMatrix : 
 [[3, 5], [5, 3]]

EnergyVector : 
 [1, 2]

Population:

[0, 1, 1, 1, 1]
[0, 1, 1, 0, 0]
[1, 1, 0, 0, 1]
[1, 0, 1, 1, 0]
[1, 1, 0, 0, 0]
[1, 0, 0, 0, 0]
[1, 1, 1, 0, 1]
[0, 1, 0, 1, 0]
[1, 1, 1, 1, 0]
[1, 1, 1, 1, 0]

MinEnergy : 
 97 with lattice [1, 0, 

In [34]:
#Test Other setup
u = [1,2,3]
N = 11
t = [[10,4,1],[4,10,5],[1,5,10]]
M = 3
generations = 10
crossoverfraction = 0.2

Pop = Population(populationSize = P,
                 latticeLength = N,
                 numParticleType = M,
                 selfEnergyVector = u,
                 interactionEnergyMatrix = t,
                 crossoverFraction = crossoverfraction)

for _ in range(generations):
    Pop.crossover()
    Pop.evaluateFitness()
    print(Pop)


------------------------------
Population [INFO]

InteractionMatrix : 
 [[10, 4, 1], [4, 10, 5], [1, 5, 10]]

EnergyVector : 
 [1, 2, 3]

Population:

[0, 2, 2, 2, 0, 0, 0, 2, 0, 2, 0]
[0, 1, 0, 2, 2, 2, 0, 0, 1, 0, 1]
[1, 2, 1, 1, 1, 0, 2, 0, 2, 1, 2]
[0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]
[2, 2, 1, 2, 0, 2, 1, 1, 0, 0, 0]
[0, 1, 2, 0, 0, 0, 1, 2, 0, 1, 2]
[1, 0, 1, 0, 1, 1, 2, 0, 1, 0, 1]
[1, 1, 1, 2, 2, 0, 0, 2, 1, 1, 2]
[1, 2, 2, 1, 0, 1, 2, 1, 2, 1, 2]
[0, 1, 0, 1, 0, 0, 1, 2, 0, 2, 1]

MinEnergy : 
 671 with lattice [0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]


------------------------------
Population [INFO]

InteractionMatrix : 
 [[10, 4, 1], [4, 10, 5], [1, 5, 10]]

EnergyVector : 
 [1, 2, 3]

Population:

[2, 0, 2, 1, 0, 0, 2, 0, 1, 2, 2]
[0, 1, 0, 2, 2, 2, 0, 0, 1, 0, 1]
[1, 2, 1, 1, 1, 0, 2, 0, 2, 1, 2]
[0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]
[2, 2, 1, 2, 0, 2, 1, 1, 0, 0, 0]
[0, 1, 2, 0, 0, 0, 1, 2, 0, 1, 2]
[1, 0, 1, 0, 1, 1, 2, 0, 1, 0, 1]
[1, 1, 1, 2, 2, 0, 0, 2, 1, 1, 2]
[1, 2, 2, 1, 0, 1,

In [35]:
#Test conduct tournament.
Pop.conductTournament()
Pop.evaluateFitness()
print(Pop)


------------------------------
Population [INFO]

InteractionMatrix : 
 [[10, 4, 1], [4, 10, 5], [1, 5, 10]]

EnergyVector : 
 [1, 2, 3]

Population:

[0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[1, 0, 0, 2, 0, 0, 2, 2, 1, 2, 2]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 1, 0, 1, 0, 0, 1, 2, 0, 2, 1]
[2, 2, 1, 2, 0, 2, 1, 1, 0, 0, 0]
[1, 0, 0, 2, 0, 0, 2, 2, 1, 2, 2]
[0, 1, 2, 0, 0, 0, 1, 2, 0, 1, 2]
[0, 2, 0, 2, 2, 0, 2, 2, 0, 1, 1]
[0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]

MinEnergy : 
 671 with lattice [0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]




In [36]:
#Test with mutiple generations
for _ in range(generations):
    Pop.crossover()
    Pop.evaluateFitness()
    Pop.conductTournament()
    Pop.evaluateFitness()
    print(Pop)

------------------------------
Population [INFO]

InteractionMatrix : 
 [[10, 4, 1], [4, 10, 5], [1, 5, 10]]

EnergyVector : 
 [1, 2, 3]

Population:

[0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]
[1, 0, 0, 2, 0, 0, 2, 2, 1, 2, 2]
[2, 2, 1, 2, 0, 2, 1, 1, 0, 0, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[2, 2, 1, 2, 0, 2, 1, 1, 0, 0, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]
[0, 2, 0, 2, 2, 0, 2, 2, 0, 1, 1]
[1, 0, 0, 2, 0, 0, 2, 2, 1, 2, 2]

MinEnergy : 
 671 with lattice [0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]


------------------------------
Population [INFO]

InteractionMatrix : 
 [[10, 4, 1], [4, 10, 5], [1, 5, 10]]

EnergyVector : 
 [1, 2, 3]

Population:

[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]
[0, 2, 0, 2, 2, 0, 2, 2, 0, 1, 1]
[0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[1, 0, 0, 2, 0, 0,

In [37]:
#Final Result
print(Pop)

------------------------------
Population [INFO]

InteractionMatrix : 
 [[10, 4, 1], [4, 10, 5], [1, 5, 10]]

EnergyVector : 
 [1, 2, 3]

Population:

[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]
[0, 2, 0, 1, 2, 2, 0, 0, 0, 1, 2]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]
[0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]

MinEnergy : 
 671 with lattice [0, 2, 1, 0, 0, 0, 2, 2, 1, 2, 0]


